In [1]:
import polars as pl
from datetime import datetime
import os

In [2]:
Depth_0 = pl.read_csv("Merged_Data/train_DEPTH_0.csv")
Depth_1 = pl.read_csv("Merged_Data/train_DEPTH_1.csv")
Depth_2 = pl.read_csv("Merged_Data/train_DEPTH_2.csv")

In [3]:
Depth_0.shape

(1526659, 225)

In [4]:
Depth_1.shape

(1221522, 85)

In [5]:
Internal_Final = Depth_0.join(
    Depth_1,
    on='case_id',
    how='left'
)

In [6]:
Internal_Final.shape

(1526659, 309)

In [ ]:
Internal_Final.columns

In [7]:
Internal_Final = Internal_Final.join(
    Depth_2,
    on='case_id',
    how='left'
)

In [8]:
Internal_Final.shape

(1526659, 320)

In [9]:
def convert_to_ordinal(date_str):
    try:
        if date_str is not None:
            return datetime.strptime(date_str, '%Y-%m-%d').date().toordinal()
        else:
            return None
    except ValueError:
        return None

Internal_Final = Internal_Final.with_columns(
    [
        pl.col("date_decision").map_elements(convert_to_ordinal).alias("date_decision"),
        pl.col("earliest_openingdate").map_elements(convert_to_ordinal).alias("earliest_openingdate")
    ]
)

numeric_str_columns = [
    "amtinstpaidbefduel24m_4187115A", "avgdbddpdlast3m_4187120P", "avgdbdtollast24m_4525197P",
    "avglnamtstart24m_4525187A", "avgoutstandbalancel6m_4187114A", "avgpmtlast12m_4525200A"
]

for col in numeric_str_columns:
    Internal_Final = Internal_Final.with_columns(pl.col(col).str.replace(",", "").cast(pl.Float64))

Internal_Final = Internal_Final.drop(["bankacctype_710L", "cardtype_51L"])

In [11]:
Internal_Final.head()

case_id,date_decision,MONTH,WEEK_NUM,target,case_id_right,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,…,unique_person_types_792L,unique_relationships_415T,unique_relationships_642T,sum_remitters,unique_roles_1084L,unique_roles_993L,sum_safeguaranty_flags,unique_sexes,unique_contact_types,unique_contact_address_districts,unique_employer_address_districts,unique_registered_address_districts,sum_is_reference_flags,unique_industries,unique_employer_zipcodes,unique_contact_zipcodes,unique_registered_zipcodes,unique_languages,unique_family_states,sum_contact_address_matchlist,sum_contact_same_employer_address,unique_person_indices,total_180dayaveragebalance,total_180dayturnover,total_30dayturnover,earliest_openingdate,max_unique_contact_type,n_unique_cacccardblochreas_147M,total_cancelled,n_unique_addres_district_368M,n_unique_addres_role_871L,n_unique_addres_zip_823M,n_unique_conts_role_79M,n_unique_empls_economicalst_849M,n_unique_empls_employedfrom_796D,n_unique_empls_employer_name_740M,n_unique_relatedpersons_role_762T
i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,737062,201901,0,0,0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,737062,201901,0,0,1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,737063,201901,0,0,2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,…,4,3,3,0,3,1,1,2,3,2,2,2,0,2,2,2,2,2,2,0,0,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3,737062,201901,0,0,3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,…,3,2,2,0,3,1,1,2,2,2,2,2,0,2,2,2,2,2,2,0,0,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4,737063,201901,0,1,4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,…,3,3,3,0,3,1,1,2,2,2,2,2,0,2,2,2,2,2,2,0,0,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
